In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from simulation import dropna
from scipy.integrate import odeint
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go
import pandas

In [2]:
master = pandas.read_csv('./fort_longer.123',sep='\t')

In [3]:
delTime = around(master['t'].diff().mean(),3)

In [4]:
master = master.fillna(0)
deltaTimeline = master['t'].diff().fillna(delTime).shift(-1)

In [5]:
master['dX'] = master['x'].diff().shift(-1)/deltaTimeline
master['dY'] = master['y'].diff().shift(-1)/deltaTimeline
master['dZ'] = master['z'].diff().shift(-1)/deltaTimeline
master = master.fillna(method='ffill')

In [40]:
master

,t,x,y,z,dX,dY,dZ
0,0.10000,0.01000,0.00001,9.50100,0.199900,0.000400,-4.980000
1,0.20000,0.02999,0.00005,9.00300,0.299900,0.000900,-4.970000
2,0.30000,0.05998,0.00014,8.50600,0.399500,0.001600,-4.960000
3,0.40000,0.09993,0.00030,8.01000,0.499200,0.002500,-4.950000
4,0.50000,0.14985,0.00055,7.51500,0.598600,0.003600,-4.940000
...,...,...,...,...,...,...,...
1994,199.50298,39.94099,-80.56109,1003.50970,3.978400,17.885000,14.960400
1995,199.60298,40.33883,-78.77259,1005.00574,3.916608,17.979902,14.968603
1996,199.70299,40.73053,-76.97442,1006.50275,3.854000,18.076800,14.980400
1997,199.80299,41.11593,-75.16674,1008.00079,3.789121,18.168283,14.988801


In [6]:
trace = go.Scatter3d(x=master['dX'],y=master['dY'],z=master['dZ'],mode='lines')
py.iplot([trace])

#### synchrony

$\phi = x_M - x_S - D \\$
$D = a_0 + a_1 sin(\omega t)$

In [41]:
a0,a1,w = 3.5,2.5,1*pi
epsilon = 1.5

def zetaInvar(stateA,stateB,t):
    return -asarray([w*a1*cos(w*t)]*len(stateA))
def chi(stateA,stateB,t):
    return stateA-stateB-a0-a1*sin(w*t)
def zetaNull(stateA,stateB,t):
    return asarray([0,0,0],dtype=float)

## slave evolution

In [42]:
def slaveEvolution(master,zeta):
    slave = zeros((len(master),3))
    stateB = master[['x','y','z']].iloc[0].to_numpy()
    #slave[0] = stateB
    for index,master_state in master.iloc[1:].iterrows():
        time = master_state['t']
        stateB = slave[index-1]
        stateA = master_state[['x','y','z']].to_numpy()

        dA = master_state[['dX','dY','dZ']].to_numpy()
        coupling = zeta(stateA,stateB,time) 
        coupling += chi(stateA,stateB,time)*epsilon
        stateB += delTime*(dA+coupling)
        slave[index] = stateB
    slave = pandas.DataFrame(slave,columns=['x','y','z'])
    return slave

In [43]:
def slaveEvolution(master,zeta):
    slave = zeros((len(master),3))
    stateB = master[['x','y','z']].iloc[0].to_numpy()
    for index,master_state in master.iloc[1:].iterrows():
        time = master_state['t']
        stateB = slave[index-1]
        stateA = master_state[['x','y','z']].to_numpy()

        dA = master_state[['dX','dY','dZ']].to_numpy()
        
        k1 = dA + chi(stateA,stateB,time) + zeta(stateA,stateB,time)
        #l1 = dA
        k2 = dA + chi(stateA + delTime*k1/2,stateB + delTime*dA/2,time + delTime/2) + zeta(stateA + delTime*k1/2,stateB + delTime*dA/2,time + delTime/2)
        #l2 = interpolated value of velocity of master at t+h/2 
        k3 = dA + chi(stateA + delTime*k2/2,stateB + delTime*dA/2,time + delTime/2) + zeta(stateA + delTime*k2/2,stateB + delTime*dA/2,time + delTime/2)
        k4 = dA + chi(stateA + delTime*k3,stateB + delTime*dA,time + delTime)

        coupling = 1/6*(k1+2*k2+2*k3+k4)
        stateB += delTime*coupling
        slave[index] = stateB
    slave = pandas.DataFrame(slave,columns=['x','y','z'])
    return slave

In [44]:
def slaveEvolution(master,zeta):
    def slaving(slave,t):
        stateA = master.loc[int(t/delTime-1)]
        A = stateA[['x','y','z']].to_numpy()
        dA = stateA[['dX','dY','dZ']].to_numpy()
        dSlave =  dA + zeta(A,slave,t) + epsilon*chi(A,slave,t)
        return dSlave
    slave0 = zeros((3))
    timeline = master['t'].to_numpy()
    slave = odeint(slaving, slave0, timeline, rtol=1.49012e-6, atol=1.49012e-6)
    slave = pandas.DataFrame(slave,columns=['x','y','z'])
    return slave

In [45]:
slave_zeta = slaveEvolution(master,zetaInvar)
slave_null = slaveEvolution(master,zetaNull)
D = a0+a1*sin(w*master['t'])
slave_truth = master[['x','y','z']] - expand_dims(D,axis=1)

In [46]:
master_trace = go.Scatter3d(name='master',x=master['x'],y=master['y'],z=master['z'],mode='lines')
slave_zeta_trace = go.Scatter3d(name='slave_zeta',x=slave_zeta['x'],y=slave_zeta['y'],z=slave_zeta['z'],mode='lines')
slave_null_trace = go.Scatter3d(name='slave_null',x=slave_null['x'],y=slave_null['y'],z=slave_null['z'],mode='lines')
slave_truth_trace = go.Scatter3d(name='true',x=slave_truth['x'],y=slave_truth['y'],z=slave_truth['z'],mode='lines')
py.iplot([master_trace,slave_zeta_trace,slave_null_trace,slave_truth_trace])

## synchrony error

In [47]:
dimension = 'y'
py.iplot([go.Scatter(name='truth_'+dimension,x=master['t'],y=slave_truth[dimension]),
         go.Scatter(name='zeta_'+dimension,x=master['t'],y=slave_zeta[dimension].shift(0)),
         go.Scatter(name='null_'+dimension,x=master['t'],y=slave_null[dimension].shift(0))])

In [48]:
error_zeta = simulation.syncError(slave_truth[['x','y','z']].to_numpy(),slave_zeta[['x','y','z']].shift(0).to_numpy())
error_null = simulation.syncError(slave_truth[['x','y','z']].to_numpy(),slave_null[['x','y','z']].shift(0).to_numpy())

In [49]:
py.iplot([go.Scatter(name='x',x=master['t'],y=error_zeta[:,0],mode='lines'),
         go.Scatter(name='y',x=master['t'],y=error_zeta[:,1],mode='lines'),
         go.Scatter(name='z',x=master['t'],y=error_zeta[:,2],mode='lines')])

In [50]:
py.iplot([go.Scatter(name='x',x=master['t'],y=error_null[:,0],mode='lines'),
         go.Scatter(name='y',x=master['t'],y=error_null[:,1],mode='lines'),
         go.Scatter(name='z',x=master['t'],y=error_null[:,2],mode='lines')])

In [51]:
print('error zeta:',dropna(error_zeta.sum(axis=1)).mean())
print('error null:',dropna(error_null.sum(axis=1)).mean())

error zeta: 0.455434093802253
error null: 8.087864406169757


In [52]:
print(simulation.lagMinimaSyncError(slave_truth,slave_zeta,10))
print(simulation.lagMinimaSyncError(slave_truth,slave_null,10))

(0.455434093802253, 0)
(5.970993671541343, -1)


In [39]:
error_zeta_trace = go.Scatter(name='zeta',x=master['t'],y=error_zeta.sum(axis=1))
error_null_trace = go.Scatter(name='null',x=master['t'],y=error_null.sum(axis=1))
py.iplot([error_zeta_trace,error_null_trace])

--------------------